## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AMD", "AAPL", "NVDA", "SMCI", "GME"],
    interval="1d",
    start_date="2000-01-01",
    end_date="2021-02-20",
)
toolbox

## Run Mandelbrot Channel Calculation

In [7]:
result = toolbox.technical.mandelbrot_channel(historical=False, window="1m")
result
result.to_polars()

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2021-02-19,"""AAPL""",127.5741,127.643997,144.8156
2021-02-19,"""AMD""",84.379,89.580002,96.2867
2021-02-19,"""GME""",10.1433,10.1475,11.3692
2021-02-19,"""NVDA""",14.239,14.891013,15.5453
2021-02-19,"""SMCI""",30.4431,31.68,34.0473


In [9]:
result

HumblObject

id: 0667182b-9b7e-7e4e-8000-a61c859bee7f
results: {"IR":{"version":506,"dsl":{"Select":{"expr":[{"Column":"date"},{"Column":...
equity_data: {"IR":{"version":0,"dsl":{"MapFunction":{"input":{"DataFrameScan":{"df...
provider: yfinance
warnings: None
chart: None
extra: {}
context_params: {'symbol': ['AMD', 'AAPL', 'NVDA', 'SMCI', 'GME'], 'interval': '1d'...
command_params: {'window': '1mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_me...